# Ayudantía 14:  RegEx - Webservices
### Autores: Fernando Duarte (`@fdduarte`) y Benjamín Martínez (`@bimartinez`)

## Parte 1: RegEx

### Limpieza de *strings*

Deberás primero arreglar el archivo memes.txt. El formato que se debe obtener es `‘id imagen, texto superior, texto inferior’`, el problema es que las comas desaparecieron y fueron reemplazadas por expresiones que cumplen las siguientes reglas:

 - Las expresiones se encuentran entre dos símbolos $.
 - Las expresiones puede contener cualquier número y símbolo de operaciones básicas (+, -, *, /, =).
 - La cantidad y orden de los caracteres dentro de la expresión no están definidos (por eso son extrañas)
 
Debes hacerlo creando la función `arreglar_memes` que debe escribir el resultado en un archivo llamado `arreglados.txt`

In [0]:
import random
import re
import requests

In [0]:
def arreglar_memes(path, path_final):
    """"
    Abre el archivo path y corrige cada una de las solicitudes del enunciado
    guardándolas en path_final
    :param path: str
    :param pathfinal: str
    :return:
    """
    new_lines = []
    with open(path, 'r') as bad_memes:
        for line in bad_memes:
            # Sustituimos todas las expresiónes en la línea por una coma
            fixed = re.sub('\$[0-9,\*,\+,\-,\/,=]+\$', ',', line)
            new_lines.append(fixed)
    with open(path_final, 'w') as ok_memes:
        ok_memes.writelines(new_lines)

### Validación Correo 

Deberás crear la función `validar_correo` que reciba un correo y retorne `True` si es válido o no. Primero debe chequear a través de RegEx y luego a través de un *request* HTTP a la dirección del correo (recuerda agregar el prefijo correspondiente al protocolo), la cual debe retornar un código de estado válido (menor a 400, puedes revisar qué significa cada uno en los contenidos), retornando `True` solo si estas dos condiciones se cumplen.

In [0]:
def validar_correo(correo):
    """
    Retorna True si el formato del correo es valido y si el servidor existe,
    en otro caso retorna False
    :param correo: str
    :return: bool
    """
    
    #Esta es la expresión para chequear que un correo es válido
    pattern = "(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)"
    
    # Chequeamos si se cumple el patrón
    if bool(re.match(pattern, correo)):
        try:
            # Aquí obtenemos el dominio del correo (el texto después de @)
            url = 'http://' + re.split('\@', correo)[1]
            
            # Realizamos la request
            response = requests.get(url)
            response = response.status_code
            if int(response) < 400:
                return True
        except requests.exceptions.ConnectionError:
            return False


## Parte 2: Webservices

### Obtener mejores meme

Crear la función `obtener_mejores` capaz de obtener los memes más populares y elegir aleatoriamente 10 de estos memes para ser mostrados.


In [0]:
def obtener_mejores():
    """
    Retorna una muestra aleatoria de 10 memes provenientes de los 100 mejores
    :return: [dict]
    """
    
    # Realizamos la request
    response = requests.get('https://api.imgflip.com/get_memes')
    
    # Obtenemos la respuesta de la API
    response = response.json()
    memes_list = response['data']['memes']
    
    # Obtenemos 10 memes al azar
    top_10 = random.sample(memes_list, 10)
    return top_10

### Crear meme

Crear la función `generar_meme` que dados el id de un meme, texto superior e inferior cree y retorne el meme creado.

In [0]:

def generar_meme(template_id, username, password, text0='', text1=''):
    """
    Genera un meme correspondiente al template_id con texto superior text0
    y texto inferior text1. Retorna la url al meme
    :param template_id: int
    :param username: str
    :param password: str
    :param text0: str
    :param text1: str
    :return: str
    """
    
    # Armamos los datos que enviaremos a la API
    data = {'template_id': template_id, 'username': username,
            'password': password, 'text0': text0, 'text1': text1}
    
    # Realizamos la request
    response = requests.post('https://api.imgflip.com/caption_image',
                             data=data)
    
    # Obtenemos la respuesta
    response = response.json()
    url = response['data']['url']
    return url